In [ ]:
import comyx
from comyx.network import BaseStation, UserEquipment, Link
from comyx.propagation import get_noise_power, get_pathloss
from comyx.utils import dbm2pow, pow2dbm, db2pow, pow2db, get_distance, generate_seed

import numpy as np
import matplotlib.pyplot as plt

# System configuration
Pt = np.linspace(10, 50, 10)  # Transmit power in dBm (10 to 50 dBm)
bandwidth = 100e6  # Bandwidth in Hz
frequency = 2.4e9  # Carrier frequency
temperature = 300  # Kelvin (room temperature)
n_RX = 2  # Number of receive antennas (BS)
n_TX = 1  # Number of transmit antennas (UE)
tones = 64  # Number of subcarriers
realizations = 1000  # Number of Monte Carlo simulations

# Calculate noise power
sigma2_dbm = get_noise_power(temperature, bandwidth, 6)  # dBm
sigma2 = dbm2pow(sigma2_dbm)  # Convert noise power to linear scale

# Initialize the system
seed = 42
fading_args = {"type": "rayleigh", "sigma": np.sqrt(1 / 2)}
pathloss_args = {
    "type": "reference",
    "alpha": 3.5,
    "p0": 30,
    "frequency": frequency,
}  # p0 is the reference power in dBm

# Initialize base station and user equipment
ax_point = BaseStation("AP", n_TX, position=[0, 0, 3])
usr_1 = UserEquipment("UE1", n_RX, position=[3, 0, 1])
usr_2 = UserEquipment("UE2", n_RX, position=[0, 3, 1])
usr_3 = UserEquipment("UE3", n_RX, position=[3, 3, 1])

# Initialize links
link_1 = Link(
    ax_point,
    usr_1,
    fading_args,
    pathloss_args,
    sigma2=sigma2,
    realizations=realizations,
    seed=seed,
)

link_2 = Link(
    ax_point,
    usr_2,
    fading_args,
    pathloss_args,
    sigma2=sigma2,
    realizations=realizations,
    seed=seed,
)

link_3 = Link(
    ax_point,
    usr_3,
    fading_args,
    pathloss_args,
    sigma2=sigma2,
    realizations=realizations,
    seed=seed,
)

# Calculate distances
distance_1 = get_distance(ax_point.position, usr_1.position)
distance_2 = get_distance(ax_point.position, usr_2.position)
distance_3 = get_distance(ax_point.position, usr_3.position)

# Pathloss calculations
pathloss_1 = get_pathloss(distance_1, pathloss_args)
pathloss_2 = get_pathloss(distance_2, pathloss_args)
pathloss_3 = get_pathloss(distance_3, pathloss_args)

# Sum-rate calculation function
def calculate_sum_rate(link, Pt_dBm):
    Pt_lin = dbm2pow(Pt_dBm)  # Convert transmit power to linear scale
    sum_rate = 0
    
    for i in range(realizations):
        H = link.get_channel_matrix(i)
        noise = link.sigma2
        snr = Pt_lin / noise  # Signal-to-noise ratio
        rate = np.log2(1 + snr * np.linalg.norm(H, ord='fro')**2)
        sum_rate += rate
    
    avg_sum_rate = sum_rate / realizations
    return avg_sum_rate

# Evaluate performance for different transmit powers
sum_rates_1 = []
sum_rates_2 = []
sum_rates_3 = []

for Pt_dBm in Pt:
    sum_rate_1 = calculate_sum_rate(link_1, Pt_dBm)
    sum_rate_2 = calculate_sum_rate(link_2, Pt_dBm)
    sum_rate_3 = calculate_sum_rate(link_3, Pt_dBm)
    
    sum_rates_1.append(sum_rate_1)
    sum_rates_2.append(sum_rate_2)
    sum_rates_3.append(sum_rate_3)
    print(f"Transmit Power: {Pt_dBm:.2f} dBm | Sum-Rate UE1: {sum_rate_1:.2f} | Sum-Rate UE2: {sum_rate_2:.2f} | Sum-Rate UE3: {sum_rate_3:.2f}")

# Plot the sum-rates versus transmit power
plt.figure(figsize=(10, 6))
plt.plot(Pt, sum_rates_1, 'b-o', label="UE1")
plt.plot(Pt, sum_rates_2, 'g-s', label="UE2")
plt.plot(Pt, sum_rates_3, 'r-^', label="UE3")
plt.xlabel('Transmit Power (dBm)')
plt.ylabel('Sum-Rate (bps/Hz)')
plt.title('Sum-Rate vs Transmit Power for Multiple UEs')
plt.grid(True)
plt.legend()
plt.show()